# sc3nb – SuperCollider3 for python / jupyter notebooks

&copy; 2017-2021 by Thomas Hermann,  <br>contributions from: Ferdinand Schlatt, Fabian Kaupmann, Dennis Reinsch, Micha Steffen Vosse

* * *


sc3nb is a python package that offers an interface to SuperCollider3. 

The goal of sc3nb is to facilitate the programming of auditory displays and interactive sonifications by
- teaming up python (and particularly numpy, scipy, pandas, matplotlib etc.) for data science
- and SuperCollider3 for interactive real-time sound rendering.


It currently allows to..

- start and control the SuperCollider audio server (scsynth)
  - The SuperCollider audio server can be started and addressed via a python SuperCollider Server implementation or via OSC messages directly.
  - There are many Python implementations of Classes from SuperCollider like `Synth`, `SynthDef`, `Buffer` and `Bus`

- use the SuperCollider language (sclang) interactively via a subprocess
  - You can write SuperCollider language code in Jupyter Notebooks and let sclang evaluate it.
  - Inject python variables into your sclang code and even get the results of the sclang code in python




sc3nb can be used for

- multi-channel audio processing
- auditory display and sonification
- sound synthesis experiment
- audio applications in general such as games or GUI-enhancements
- signal analysis and plotting
- computer music and just in time music control
- any usecase that the SuperCollider 3 language supports


It is meant to grow into a backend for a sonification package, and can be used both from jupyter and in standard python software development.

sc3nb is hosted at GitHub https://github.com/thomas-hermann/sc3nb

## Getting Started

### Installing sc3nb

- To use sc3nb you need a installation of SuperCollider on your system. See https://supercollider.github.io/download for instructions.
- To install sc3nb you can
  - install it locally in editable mode (i.e. changes to sc3nb code will automatically be "re-installed"). 
    - clone the repository from https://github.com/thomas-hermann/sc3nb 
    -  from inside the sc3nb directory run `pip install -e .`
  - or install it directly from GitHub using `pip install git+git://github.com/thomas-hermann/sc3nb@master`

However we are currently making sure that sc3nb can also be installed via `pip install sc3nb` from PyPI

### Starting sc3nb

In [1]:
import sc3nb as scn

To startup sc3nb (sclang, scsynth, and a python OSC server) use `startup`

In [2]:
sc = scn.startup()  # optionally specify arg sclang_path="/path/to/sclang"

<IPython.core.display.Javascript object>

Starting sclang process...


Done.
Registering OSC /return callback in sclang...
Done.
Loading default SynthDescs
Done.
Booting SuperCollider Server...


Done.


You can produce a test sound with `blip`, which should relax any tensions whether the server is up and running. This sound should be played by the default server start.

In [3]:
sc.server.blip()

`sc` provides you the interfaces for

* `scsynth` via `sc.server`
* `sclang` via `sc.lang`

**Configuration of sc3nb startup**:

- If the executables are not in `$PATH`, you can specify them with `sclang_path` / `scsynth_path` f.e. `sclang_path="/path/to/sclang-containing-dir/sclang"`

  - On macOS they reside in `/Applications/SuperCollider.app/Contents` in the folders MacOS and Resources. <br>
  To add these paths to your `$PATH`, simply add to your ~/.profile, e.g. (please adapt to your installation): <br>
  `PATH=$PATH:/Applications/SuperCollider.app/Contents/MacOS:/Applications/SuperCollider.app/Contents/Resources`  
  - On Windows they reside in your Installation folder f.e `C:\Program Files\SuperCollider-3.x.x`

* You could also only use **scsynth** and don't start **sclang** with `start_sclang=False`
* See `help(scn.startup)` for all options

## Basic examples

Create and control SuperCollider Synths

In [4]:
syn = scn.Synth()
syn

<Synth(20001) 'default' s {}>

In [5]:
syn.freq = 800
syn

<Synth(20001) 'default' s {'freq': 800}>

In [6]:
syn.free()

<Synth(20001) 'default' sf {'freq': 800}>

Send SynthDefs with python code injection

In [7]:
synth_dur = 0.5  # you can use python variable values in SynthDefs
synth_def = scn.SynthDef('random', """{ |out|
Out.ar(out, SinOsc.ar(Rand(400, 800), 0, 0.2) * Line.kr(1, 0, ^synth_dur, doneAction: Done.freeSelf))
}""")
synth_def.add()

scn.Synth("random")

<Synth(20002) 'random' s {}>

Load File as Buffer and play it

In [8]:
buf = scn.Buffer().read("./media/blip.wav")
buf

Buffer 128 on ('127.0.0.1', 57110): 2 x 43008 @ 44100 Hz –> allocated using mode 'file'

In [9]:
buf.play()

<Synth(20003) 'sc3nb_playbuf_128' s {}>

In [10]:
buffer_data = buf.to_array()
print(buffer_data.shape)
buffer_data

(43008, 2)


array([[-0.00189209, -0.00189209],
       [-0.00430298, -0.00430298],
       [-0.00708008, -0.00708008],
       ...,
       [ 0.19314575,  0.19314575],
       [ 0.18963623,  0.18963623],
       [ 0.18585205,  0.18585205]])

Create OSC Bundles

In [11]:
with sc.server.bundler() as bundler:
    scn.Synth("random")
    bundler.wait(0.2)
    scn.Synth("random")
    bundler.wait(0.5)

Execute SuperCollider Language Code

In [12]:
breakfast = "eggs"
sc.lang.cmdv('^breakfast.scramble;')

-> sgge


Get results from SuperCollider Language in python

In [13]:
x = 5
value = %scg (1..^x)
print(f"received {value} with type {type(value)}")

-> [ 1, 2, 3, 4, 5 ]
received [1, 2, 3, 4, 5] with type <class 'list'>


OSC messaging : f.e. free sclang Synth with OSC message

In [14]:
nodeid = %scg x = Synth("default"); x.nodeID;

-> 1000


In [15]:
sc.server.msg("/n_free", nodeid)

## Further informations

For more details on the specific parts please refer to the User Guide.

- [SuperCollider Language **sclang**](./sclang-examples.ipynb)

- Usage of the SuperCollider Objects python interface
  - [Server](./supercollider-objects/server-examples.ipynb)
  - [Nodes (Synth and Group)](./supercollider-objects/node-examples.ipynb)
  - [SynthDef](./supercollider-objects/synthdef-examples.ipynb)
  - [Buffer](./supercollider-objects/buffer-examples.ipynb)
  - [Bus](./supercollider-objects/bus-examples.ipynb)
  - [Recorder](./supercollider-objects/recorder-examples.ipynb)


- Usage of common SuperCollider functions [Helpers](./helper-examples.ipynb)
- Usage of OSC in SuperCollider: [OSC Communication](./osc-communication-example.ipynb)

## How does this work?

- **sclang** and **scsynth** are started as subprocesses and thier outputs is collected. 
- **scsynth** is then controlled via OSC
  * Direct control of the server shortcuts a detour via sclang and is both more efficient and promises a lower latency
- **sclang** communication is done with pipes and OSC for receiving return values



## Caveats and Problems

We strongly encourage you to share any problems that arise using sc3nb with us.

There are some things to consider when using sc3nb.


- The shortcut `ctrl/cmd + .` does currently only works in classic Jupyter notebooks not in JupyterLab. It also will import sc3nb as `sc3nb` and thus you should avoid variables with the name `sc3nb`.
- We depend on the output to stdout of sclang and scsynth in some cases like the startup of the server or more obviously for the sclang output when using cmd with verbosity. This means if some language settings or updates of sclang/scsynth change the outputs to something that we don't expect stuff will fail. However this should be quite easy to patch.
- There is a bug in Jupyter (https://github.com/jupyter/jupyter_client/issues/104) that does leave sclang and scsynth running when restarting the Jupyter kernel. However calling `exit()` on the SC instance will kill them. To aviod conflicts with orphaned sclangs/scsynths we also look for leftover sclang/scsynth processes on the start of each and will try to kill them. This might lead to killing sclang or scsynth processes that you wanted to keep. You can specify which parent processes are allowed for sclang/scsynth processes with the `allowed_parents` paramter.
- The SuperCollider Objects are currently not guaranteed to be thread safe.
- Another thing about SuperCollider in general is to keep in mind that there are limits in the network communication. Currently we only support UDP and therefore have the corresponding limits. As an example a Bundler will fail when you try to send too many messages.

## Contributing

Please get in touch with us if you wish to contribute. We are happy to be involved in the discussion of new features and to receive pull requests.

### Setting up the development environment

Additional dependencies for development and building docs, etc. can be installed via the following


| Install                                | Purpose            |
|:---------------------------------------|:-------------------|
| `pip install -r requirements_dev.txt`  | making commits     |
| `pip install -r requirements_test.txt` | executing tests    |
| `pip install -r requirements_doc.txt`  | building the docs  |


### Development Guidelines

[![pre-commit](https://img.shields.io/badge/pre--commit-enabled-brightgreen?logo=pre-commit&logoColor=white)](https://pre-commit.com/)
[![Code style: black](https://img.shields.io/badge/code%20style-black-000000.svg)](https://github.com/psf/black)

* Use [black](https://github.com/psf/black) formatting
* Use [python type hints](https://docs.python.org/3/library/typing.html)
* [numpydoc](https://numpydoc.readthedocs.io/en/latest/example.html) docstring format. Also see [How to Document](https://numpy.org/doc/stable/docs/howto_document.html)
* To ensure standards are followed we use [pre-commit](https://pre-commit.com/) 


## Useful Ressources

If you are looking for more information about SuperCollider you should checkout the following links

- [SuperCollider website](https://supercollider.github.io/)
- [SuperCollider on GitHub](https://github.com/supercollider/supercollider)


- [SuperCollider documentation and help](https://doc.sccode.org/)
  - [Client vs Server](https://doc.sccode.org/Guides/ClientVsServer.html)
  - [Server Architecture - Main Design Concepts](https://doc.sccode.org/Reference/Server-Architecture.html)
  - [Server Command Reference](https://doc.sccode.org/Reference/Server-Command-Reference.html)
  - [Server Guide](https://doc.sccode.org/Guides/Server-Guide.html)
  - [Multi-client Setups](https://doc.sccode.org/Guides/MultiClient_Setups.html)
  - [OSC Communication](https://doc.sccode.org/Guides/OSC_communication.html)


- SuperCollider Language
   - [sclang Startup File](https://doc.sccode.org/Reference/StartupFile.html)
   - [Symbolic Notations](https://doc.sccode.org/Overviews/SymbolicNotations.html)
   - [Operators](https://doc.sccode.org/Overviews/Operators.html)
   - [Syntax Shortcuts](https://doc.sccode.org/Reference/Syntax-Shortcuts.html)


In [16]:
sc.exit()  # To shut down the server and sclang subprocesses